<a href="https://colab.research.google.com/github/Carson-Thompson/Lab_6_1/blob/main/lab_6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').getOrCreate()

In [41]:
!curl https://raw.githubusercontent.com/Carson-Thompson/Lab_6_1/refs/heads/main/data/health_survey_v2.csv -o ./sample_data/health_survey.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  195k  100  195k    0     0   500k      0 --:--:-- --:--:-- --:--:--  499k


In [42]:
(health_survey :=
 spark.read.csv("./sample_data/health_survey.csv", header=True, inferSchema=True, nullValue='-')
)

DataFrame[ID: int, F1: string, F5: string, F2: string, F1_1: string, F2_1: string, F6: string, F4: string, F3: string, F5_1: string, F1_2: string, F2_2: string, F6_1: string, F2_3: string, F4_1: string, F2_4: string, F5_2: string, F2_5: string, F6_2: string, F1_3: string, F2_6: string, F5_3: string, F4_2: string, F2_7: string, F3_1: string, F2_8: string, F5_4: string, F3_2: string, F1_4: string, F3_3: string, F1_5: string, F5_5: string, F6_3: string, F1_6: string, F5_6: string, F2_9: string, F3_4: string, F4_3: string, F2_10: string, F1_7: string, F6_4: string, F4_4: string, F5_7: string, F3_5: string, F2_11: string]

In [43]:
!curl https://raw.githubusercontent.com/Carson-Thompson/Lab_6_1/refs/heads/main/data/ReverseCodingItems_v2.csv -o ./sample_data/reverse_coding.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3157  100  3157    0     0  10155      0 --:--:-- --:--:-- --:--:-- 10183


In [53]:
(reverse_coding :=
 spark.read.csv("./sample_data/reverse_coding.csv", header=True, inferSchema=True, nullValue='-')
 .withColumnRenamed("Question", "Survey Question")
)

DataFrame[Survey Question: string, Construct: int, Question # on Qualtrics Survey: int, Needs Reverse Coding?: string, Column Name: string]

In [79]:
from pyspark.sql.functions import col, when, substring, sum

survey_summary = (health_survey
                  .unpivot(ids = "ID",
                           values = health_survey.columns[1:],
                           variableColumnName = "Question",
                           valueColumnName = "Response")
                  .join(reverse_coding, on=col("Question") == col("Column Name"), how="left")
                  .drop("Survey Question", "Construct", "Question # on Qualtrics Survey", "Column Name")
                  .withColumn("TempRecodeValue",
                               when(col("Response") == "Strongly Agree", 5)
                              .when(col("Response") == "Somewhat Agree", 4)
                              .when(col("Response") == "Neither Agree nor Disagree", 3)
                              .when(col("Response") == "Somewhat Disagree", 2)
                              .when(col("Response") == "Strongly Disagree", 1)
                             )
                  .withColumn("TempReverseValue",
                               when(col("Response") == "Strongly Agree", 1)
                              .when(col("Response") == "Somewhat Agree", 2)
                              .when(col("Response") == "Neither Agree nor Disagree", 3)
                              .when(col("Response") == "Somewhat Disagree", 4)
                              .when(col("Response") == "Strongly Disagree", 5)
                             )
                  .withColumn("RecodedValue",
                               when(col("Needs Reverse Coding?") == "Yes", col("TempReverseValue"))
                              .otherwise(col("TempRecodeValue"))
                             )
                  .withColumn("Question Type", substring(col("Question"), 1, 2))
                  .drop("Question", "Response", "Needs Reverse Coding?", "TempRecodeValue", "TempReverseValue")
                  .groupby("ID", "Question Type")
                  .agg(sum("RecodedValue").alias("RecodedTotal"))
                  .groupby("ID")
                  .pivot("Question Type")
                  .agg(sum("RecodedTotal"))
                  .select("ID", "F1", "F2", "F3", "F4", "F5", "F6")
                  .orderBy("ID")
                 )

In [81]:
survey_summary.limit(5).toPandas()

,ID,F1,F2,F3,F4,F5,F6
0,1,31,48,20,17,28,18
1,2,31,47,19,17,27,20
2,3,36,46,19,18,32,17
3,4,32,54,12,15,30,16
4,5,37,47,22,19,36,19


In [87]:
survey_summary.write.csv("./sample_data/health_survey_summary_v2.csv", header=True)

In [88]:
!ls ./sample_data/health_survey_summary_v2.csv

part-00000-6aeab418-5b07-4cef-b56a-71a1327159e2-c000.csv  _SUCCESS


In [90]:
from google.colab import files
files.download("./sample_data/health_survey_summary_v2.csv/part-00000-6aeab418-5b07-4cef-b56a-71a1327159e2-c000.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>